In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 30 16:16:49 2021

@author: acmsavazzi
"""
#%% HARMONIE_plot.py

#%%                             Libraries
###############################################################################
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import os
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import sys
from datetime import datetime, timedelta
from netCDF4 import Dataset
from scipy import ndimage as ndi

my_source_dir = os.path.abspath('{}/../../../../My_source_codes')
sys.path.append(my_source_dir)
from My_thermo_fun import *
sys.path.insert(1, '/Users/acmsavazzi/Documents/WORK/PhD_Year2/Coding/Scale_separation/')
from functions import *

import pint_xarray
from pint_xarray import unit_registry as ureg


import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
proj=ccrs.PlateCarree()
coast = cartopy.feature.NaturalEarthFeature(\
        category='physical', scale='50m', name='coastline',
        facecolor='none', edgecolor='r')

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
         'axes.labelsize': 24,
         'axes.titlesize':'large',
         'xtick.labelsize':20,
         'ytick.labelsize':20,
         'figure.figsize':[10,7],
         'figure.titlesize':24}
pylab.rcParams.update(params)
import dask
dask.config.set({"array.slicing.split_large_chunks": True})

def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

def convert_rain_intensity(intensity_kg_kg):
    density_water_vapor = 0.9  # kg/m³
    conversion_factor = 1000 / 3600  # Conversion from kg/m³ to g/m³ and seconds to hours

    intensity_mm_hour = intensity_kg_kg * density_water_vapor * conversion_factor
    return intensity_mm_hour
#%% initial 
dt          = 75*ureg.seconds                 # model  timestep [seconds]
step        = 3600*ureg.seconds             # output timestep [seconds]
domain_name = 'BES'
lat_select  = 13.2806    # HALO center 
lon_select  = -57.7559   # HALO center 

domain_sml  = 200*ureg.kilometers            # km
domain_med  = 400*ureg.kilometers 
grid        = 2.5*ureg.kilometers  # kmm
srt_time    = np.datetime64('2020-01-03T00:30')
end_time    = np.datetime64('2020-01-29T23')

months = ['01',]
month='0*'
plot=False
apply_filter = False

# exps = ['noHGTQS_','noHGTQS_noSHAL_']
exps = ['noHGTQS_','noHGTQS_noUVmix_','noHGTQS_noSHAL_']
col=['k','r','g']
# col=['k','r']
sty=['--','-',':']
# sty=['--','-']

levels = 'z'      ## decide wether to open files model level (lev) or 
                    ## already interpolate to height (z)
my_harm_dir     = '/Users/acmsavazzi/Documents/WORK/PhD_Year3/DATA/HARMONIE/'
ifs_dir         = '/Users/acmsavazzi/Documents/WORK/Research/MyData/'

ModuleNotFoundError: No module named 'pint_xarray'

In [ ]:
#%% Wind tendency from shallow convective parameterisation 
exp = exps[0]
n_xplots = 3
n_yplots = 2
cmap_ = cm.coolwarm

fig, axs = plt.subplots(n_yplots,n_xplots,figsize=(11,11))
for idx, var in enumerate(['utend_conv','vtend_conv']):
    if var == 'utend_conv':
        write_title = 'Zonal tendency'
        wind_var = 'ua'
        flx_var = 'uflx_conv'
    elif var == 'vtend_conv':
        write_title = 'Meridional tendency'
        wind_var = 'va'
        flx_var = 'vflx_conv'
    ## Winds
    # harm3d[exp][wind_var].sel(z=slice(0,4000)).mean(('x','y','time'))\
    #         .plot(y='z',lw=1.5,ax=axs[idx,0],c='k',label='Mean')
    harm3d[exp][wind_var].where(filtered[exp].sel(klp=5)['smoc_conv']>0).sel(z=slice(10,4000)).mean(('x','y')).mean('time')\
            .plot(y='z',lw=2,ax=axs[idx,0],c='orangered',label=r'$D_{sc}>0$')
    harm3d[exp][wind_var].where(filtered[exp].sel(klp=5)['smoc_dive']>0).sel(z=slice(10,4000)).mean(('x','y')).mean('time')\
            .plot(y='z',lw=2,ax=axs[idx,0],c='royalblue',label=r'$D_{sc}<0$')
    harm3d[exp][wind_var].where(filtered[exp].sel(klp=5)['smoc']==0).sel(z=slice(10,4000)).mean(('x','y')).mean('time')\
            .plot(y='z',lw=3,ax=axs[idx,0],c='olive',label=r'NoSMOC')    
    
    ## Fluxes
    # harm3d[exp][flx_var].sel(z=slice(0,4000)).mean(('x','y','time'))\
    #         .plot(y='z',lw=1.5,ax=axs[idx,1],c='k',label='Mean')
    harm3d[exp][flx_var].where(filtered[exp].sel(klp=5)['smoc_conv']>0).sel(z=slice(0,4000)).mean(('x','y')).mean('time')\
            .plot(y='z',lw=2,ax=axs[idx,1],c='orangered',label=r'$D_{sc}>0$')
    harm3d[exp][flx_var].where(filtered[exp].sel(klp=5)['smoc_dive']>0).sel(z=slice(0,4000)).mean(('x','y')).mean('time')\
            .plot(y='z',lw=2,ax=axs[idx,1],c='royalblue',label=r'$D_{sc}<0$')
    harm3d[exp][flx_var].where(filtered[exp].sel(klp=5)['smoc']==0).sel(z=slice(0,4000)).mean(('x','y')).mean('time')\
            .plot(y='z',lw=3,ax=axs[idx,1],c='olive',label=r'NoSMOC')

    ## Tendency 
    ## Tendency normalised by the wind
    # (harm3d[exp][var].sel(z=slice(0,4000)).mean(('x','y','time'))\
    #     /harm3d[exp][wind_var].sel(z=slice(0,4000)).mean(('x','y','time')))\
    #         .plot(y='z',lw=1.5,ax=axs[idx,2],c='k',label='Mean')
    (harm3d[exp][var].where(filtered[exp].sel(klp=5)['smoc_conv']>0).sel(z=slice(0,4000)).mean(('x','y')).mean('time')\
        /harm3d[exp][wind_var].where(filtered[exp].sel(klp=5)['smoc_conv']>0).sel(z=slice(0,4000)).mean(('x','y')).mean('time'))\
            .plot(y='z',lw=2,ax=axs[idx,2],c='orangered',label=r'$D_{sc}>0$')
    (harm3d[exp][var].where(filtered[exp].sel(klp=5)['smoc_dive']>0).sel(z=slice(0,4000)).mean(('x','y')).mean('time')\
        /harm3d[exp][wind_var].where(filtered[exp].sel(klp=5)['smoc_dive']>0).sel(z=slice(0,4000)).mean(('x','y')).mean('time'))\
            .plot(y='z',lw=2,ax=axs[idx,2],c='royalblue',label=r'$D_{sc}<0$')
    (harm3d[exp][var].where(filtered[exp].sel(klp=5)['smoc']==0).sel(z=slice(0,4000)).mean(('x','y')).mean('time')\
        /harm3d[exp][wind_var].where(filtered[exp].sel(klp=5)['smoc']==0).sel(z=slice(0,4000)).mean(('x','y')).mean('time'))\
            .plot(y='z',lw=3,ax=axs[idx,2],c='olive',label=r'NoSMOC')
    ##
    axs[idx,1].set_yticks([])
    axs[idx,1].set_ylabel('')
    axs[idx,2].set_yticks([])
    axs[idx,2].set_ylabel('')
    for idy in [0,1,2]:
        axs[idx,idy].axvline(0,c='grey',lw=0.5)
        axs[idx,idy].set_ylim([0,4000])
        axs[idx,idy].axhspan(sc_layer_base, sc_layer_top, alpha=0.1, color='grey')
        axs[idx,idy].axhspan(c_layer_base, c_layer_top, alpha=0.1, color='grey')
    axs[idx,2].set_xlim([-0.04,0.03])
    axs[idx,2].set_xticks(np.linspace(-0.03, 0.03, 3))
    axs[idx,2].set_title(write_title,fontsize=22)
    ##
axs[0,0].set_xlim([None,-1.5])
axs[1,0].set_xlim([None,-0.3])
axs[0,0].set_xlabel('')
axs[0,1].set_xlabel('')
axs[0,2].set_xlabel('')
axs[0,0].set_title('Zonal wind',fontsize=20)
axs[1,0].set_title('Meridional wind',fontsize=20)
axs[0,1].set_title('Zonal flux',fontsize=20)
axs[1,1].set_title('Meridional flux',fontsize=20)
axs[idx,0].set_xlabel(r'$m s^{-1}$')
axs[idx,1].set_xlabel(r'$m^2 s^{-2}$')
axs[idx,2].set_xlabel(r'$hour^{-1}$')
axs[0,0].legend(fontsize=14)